In [ ]:
import numpy as np
import pandas as pd
import optuna
import matplotlib.pyplot as plt
import networkx as nx
import itertools
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from xgboost import XGBClassifier

In [ ]:
# Cancer data
stage_1_samples = pd.read_csv('../data/cancer/stage_1_prostate_cancer_samples.csv')
stage_1_samples.head(2)

In [ ]:
stage_2_samples = pd.read_csv('../data/cancer/stage_2_prostate_cancer_samples.csv')
stage_2_samples.head(2)

In [ ]:
# Combine stage 1 and stage 2 samples
combined_dataset = pd.concat([stage_1_samples, stage_2_samples], ignore_index=True)

# Verify the Stage column exists and print unique values
print("Unique values in Stage column:", combined_dataset['Stage'].unique())

# Strip the "Stage: " prefix from the Stage column and then label the samples
combined_dataset['Stage'] = combined_dataset['Stage'].str.strip()

# Label the samples (0 for stage 1, 1 for stage 2)
combined_dataset['ID_REF'] = np.where(combined_dataset['Stage'] == 'Stage: 1', 0, 1)
combined_dataset['ID_REF'] = np.where(combined_dataset['Stage'] == 'Stage: 2', 1, combined_dataset['ID_REF'])

# Print class distribution to ensure both classes are present
print("Class distribution in ID_REF column:")
print(combined_dataset['ID_REF'].value_counts()) 

In [ ]:
# Define process_data function
def process_data(data, under_sample_factor=None, over_sample_factor=None):
    # Drop metadata columns
    columns_to_drop = ['Sample_ID', 'Sex', 'Age', 'Stage', 'Disease']
    data = data.drop(columns=columns_to_drop, axis=1)
    
    x = np.array(data.drop(["ID_REF"], axis=1)).astype('float')
    y = np.array(data["ID_REF"]).astype('int')
    feature_names = data.columns[1:]

    if under_sample_factor is not None and isinstance(under_sample_factor, float) and 0 < under_sample_factor <= 1:
        under_sampler = RandomUnderSampler(sampling_strategy=under_sample_factor)
        x, y = under_sampler.fit_resample(x, y)

    if over_sample_factor is not None and isinstance(over_sample_factor, float) and 0 < over_sample_factor <= 1:
        over_sampler = RandomOverSampler(sampling_strategy=over_sample_factor)
        x, y = over_sampler.fit_resample(x, y)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0, stratify=y)

    # Normalization
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    return x_train, x_test, y_train, y_test, feature_names

In [ ]:
# Define parameters
feature_selection_num = 500
feature_importance_num = 10

# Process data
x_train, x_test, y_train, y_test, feature_names = process_data(combined_dataset, under_sample_factor=None, over_sample_factor=None)

In [ ]:
# Define the objective function for Optuna - SVM
def svm_objective(trial):
    k = feature_selection_num
    
    # Suggest hyperparameters for SVM
    C = trial.suggest_loguniform('C', 1e-6, 1e+6)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    
    # Create the pipeline
    pipe = Pipeline([
        ('skb', SelectKBest(f_classif, k=k)),
        ('estimator', SVC(C=C, kernel=kernel))
    ])
    
    # Perform cross-validation
    cv = StratifiedKFold(n_splits=10)
    scores = cross_val_score(pipe, x_train, y_train, cv=cv, scoring='accuracy')
    return scores.mean()

In [ ]:
# Define the objective function for Optuna - Random Forest
def rf_objective(trial):
    k = feature_selection_num
    
    # Suggest hyperparameters for Random Forest
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    
    # Create the pipeline
    pipe = Pipeline([
        ('skb', SelectKBest(f_classif, k=k)),
        ('estimator', RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features, criterion=criterion))
    ])
    
    # Perform cross-validation
    cv = StratifiedKFold(n_splits=10)
    scores = cross_val_score(pipe, x_train, y_train, cv=cv, scoring='accuracy')
    return scores.mean()

In [ ]:
# Define the objective function for Optuna - XGBoost
def xgb_objective(trial):
    k = feature_selection_num
    
    # Suggest hyperparameters for XGBoost
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e+1)
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    
    # Create the pipeline
    pipe = Pipeline([
        ('skb', SelectKBest(f_classif, k=k)),
        ('estimator', XGBClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth))
    ])
    
    # Perform cross-validation
    cv = StratifiedKFold(n_splits=10)
    scores = cross_val_score(pipe, x_train, y_train, cv=cv, scoring='accuracy')
    return scores.mean()

In [ ]:
# Optimize hyperparameters using Optuna
svm_study = optuna.create_study(direction='maximize')
svm_study.optimize(svm_objective, n_trials=100)

In [ ]:
# Random Forest with hyperparameter optimization
rf_study = optuna.create_study(direction='maximize')
rf_study.optimize(rf_objective, n_trials=100)

In [ ]:
# # XGBoost with hyperparameter optimization
# xgb_study = optuna.create_study(direction='maximize')
# xgb_study.optimize(xgb_objective, n_trials=1)

In [ ]:
# Print best trial for each study
print("Best SVM trial:")
svm_trial = svm_study.best_trial
print("  Value: ", svm_trial.value)
print("  Params: ")
for key, value in svm_trial.params.items():
    print(f"    {key}: {value}")

print("Best Random Forest trial:")
rf_trial = rf_study.best_trial
print("  Value: ", rf_trial.value)
print("  Params: ")
for key, value in rf_trial.params.items():
    print(f"    {key}: {value}")

# print("Best XGBoost trial:")
# xgb_trial = xgb_study.best_trial
# print("  Value: ", xgb_trial.value)
# print("  Params: ")
# for key, value in xgb_trial.params.items():
#     print(f"    {key}: {value}")

In [ ]:
# Analysis
def create_network(top_features_list, all_features_list, correlation_threshold_factor, cancer_dataset):
    cancer_subset, control_subset = cancer_dataset[(cancer_dataset["ID_REF"] == 0)], cancer_dataset[(cancer_dataset["ID_REF"] == 1)]

    edges = [((node1, node2), cancer_subset[node1].corr(cancer_subset[node2], method="pearson")) for node1, node2 in itertools.combinations(top_features_list, 2)]
    edges = [(node1, node2, {'weight': abs(correlation), 'sign': 1 if correlation > 0 else 0}) for (node1, node2), correlation in edges if abs(correlation) >= correlation_threshold_factor]

    nodes = [(feature, {'sides': all_features_list.count(feature) + 1, "comparison": 1 if cancer_subset[feature].mean() >= control_subset[feature].mean() else (0 if cancer_subset[feature].mean() < control_subset[feature].mean() else 0.5)}) for feature in top_features_list]

    graph = nx.Graph()
    graph.add_nodes_from(nodes)
    graph.add_edges_from(edges)

    degrees = dict(graph.degree())
    network_degrees_values, network_degrees_nodes = zip(*sorted(zip(degrees.values(), degrees.keys()), reverse=True))
    print(network_degrees_nodes)
    print(network_degrees_values)

    return graph

In [ ]:
# Train and evaluate the models with the best hyperparameters
def train_and_evaluate(pipe, x_train, y_train, x_test, y_test):
    pipe.fit(x_train, y_train)
    y_pred = pipe.predict(x_test)
    print("Testing Accuracy:", accuracy_score(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

In [ ]:
# SVM
best_svm_params = svm_trial.params
svm_pipe = Pipeline([
    ('skb', SelectKBest(f_classif, k=feature_selection_num)),
    ('estimator', SVC(**best_svm_params))
])
train_and_evaluate(svm_pipe, x_train, y_train, x_test, y_test)


In [ ]:
# Random Forest
best_rf_params = rf_trial.params
rf_pipe = Pipeline([
    ('skb', SelectKBest(f_classif, k=feature_selection_num)),
    ('estimator', RandomForestClassifier(**best_rf_params))
])
train_and_evaluate(rf_pipe, x_train, y_train, x_test, y_test)

In [ ]:
# # XGBoost
# best_xgb_params = xgb_trial.params
# xgb_pipe = Pipeline([
#     ('skb', SelectKBest(f_classif, k=feature_selection_num)),
#     ('estimator', XGBClassifier(**best_xgb_params))
# ])
# train_and_evaluate(xgb_pipe, x_train, y_train, x_test, y_test)

In [ ]:
# Feature importance and top features can be extracted similarly to the previous script

def get_top_features(pipe, feature_names, top_feature_num):
    if isinstance(pipe.named_steps['estimator'], SVC):
        feature_scores = pipe.named_steps['estimator'].coef_[0]
    elif isinstance(pipe.named_steps['estimator'], RandomForestClassifier) or isinstance(pipe.named_steps['estimator'], XGBClassifier):
        feature_scores = pipe.named_steps['estimator'].feature_importances_
    features = pipe.named_steps['skb'].get_support(indices=True)
    top_indices = np.argsort(np.abs(feature_scores))[::-1][:top_feature_num]
    top_features = [(feature_names[i], feature_scores[i]) for i in top_indices]
    return top_features

In [ ]:
# Get top features for each model
svm_top_features = get_top_features(svm_pipe, feature_names, feature_importance_num)
rf_top_features = get_top_features(rf_pipe, feature_names, feature_importance_num)
# xgb_top_features = get_top_features(xgb_pipe, feature_names, feature_importance_num)

In [ ]:
print("Top SVM features:", svm_top_features)
print("Top Random Forest features:", rf_top_features)
# print("Top XGBoost features:", xgb_top_features)

In [ ]:
# Analysis and visualization
def create_network(top_features_list, all_features_list, correlation_threshold_factor, cancer_dataset):
    cancer_subset, control_subset = cancer_dataset[(cancer_dataset["ID_REF"] == 1)], cancer_dataset[(cancer_dataset["ID_REF"] == 0)]

    edges = [((node1, node2), cancer_subset[node1].corr(cancer_subset[node2], method="pearson")) for node1, node2 in itertools.combinations(top_features_list, 2)]
    edges = [(node1, node2, {'weight': abs(correlation), 'sign': 1 if correlation > 0 else 0}) for (node1, node2), correlation in edges if abs(correlation) >= correlation_threshold_factor]

    nodes = [(feature, {'sides': all_features_list.count(feature) + 1, "comparison": 1 if cancer_subset[feature].mean() >= control_subset[feature].mean() else (0 if cancer_subset[feature].mean() < control_subset[feature].mean() else 0.5)}) for feature in top_features_list]

    graph = nx.Graph()
    graph.add_nodes_from(nodes)
    graph.add_edges_from(edges)

    degrees = dict(graph.degree())

    network_degrees_values, network_degrees_nodes = zip(*sorted(zip(degrees.values(), degrees.keys()), reverse=True))
    print(network_degrees_nodes)
    print(network_degrees_values)

    return graph

In [ ]:
def create_bar_charts(top_features_list, full_dataset, path_name):
    lung_cancer = full_dataset[(full_dataset["ID_REF"] == "Lung Cancer")]
    colorectal_cancer = full_dataset[(full_dataset["ID_REF"] == "Colorectal Cancer")]
    gastric_cancer = full_dataset[(full_dataset["ID_REF"] == "Gastric Cancer")]
    prostate_cancer = full_dataset[(full_dataset["ID_REF"] == "Prostate Cancer")]
    no_cancer = full_dataset[(full_dataset["ID_REF"] == "No Cancer")]

    cancer_dataset = [lung_cancer, colorectal_cancer, gastric_cancer, prostate_cancer, no_cancer]

    plt.ioff()
    for feature_name in top_features_list:
        plt.figure(0).clf()
        plt.figure(figsize=(9, 6))

        plt.rcParams.update({'font.size': 18})

        labels = ['Lung', 'Colorectal', 'Gastric', 'Prostate', 'No Cancer']
        means = [data[feature_name].mean() for data in cancer_dataset]
        errors = [data[feature_name].sem() * 2 for data in cancer_dataset]
        plt.bar(labels, means, yerr=errors, error_kw={'elinewidth': 10, 'ecolor': 'k'}, capsize=15)

        plt.title(feature_name)
        plt.ylabel('Signal Value')
        plt.savefig("../bar_charts/" + path_name + "/" + feature_name, dpi=200, bbox_inches='tight')
    plt.ion()

In [ ]:
# # Create network and bar charts for visualization
# network_graph = create_network(top_features, list(feature_names), correlation_threshold_factor=0.5, cancer_dataset=combined_dataset)
# create_bar_charts(top_features, combined_dataset, path_name="Feature_Bar_Charts")